In [2]:
import numpy as np
import pandas as pd
import multiprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.datasets import make_classification, load_wine
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from warnings import filterwarnings
filterwarnings('ignore')

#### 나이브 베이스 분류기(Naive Bayes Classification)<br>

* 베이즈 정리를 적용한 확률적 분류 알고리즘
* 모든 특성들이 독립(Independent)임을 가정 (naive 가정)
* 입력 특성에 따라 3개의 분류기 존재
  * Gaussian Naive Bayes Classification    : int(정수), float(실수) => Continuous data
  * Bernoulli Naive Bayes Classification   : binary(이진)
  * Multinomial Naive Bayes Classification : int(정수)
  * alpha : smoothing

#### 나이브 베이즈 분류(Naïve Bayes Classification)의 확률 모델<br>

* 나이브 베이즈(Naïve Bayes Classification)는 조건부 확률 모델.
* *N*개의 특성을 나타내는 벡터 **x**를 입력 받아 k개의 가능한 확률적 결과를 출력.

\begin{equation}
p(C_k | x_1,...,x_n)
\end{equation}

* 위의 식에 베이즈 정리를 적용하면 다음과 같음.

\begin{equation}
p(C_k | \textbf{x}) = \frac{p(C_k)p(\textbf{x}|C_k)}{p(\textbf{x})}
\end{equation}

* 위의 식에서 분자만이 출력 값에 영향을 받기 때문에 분모 부분을 상수로 취급할 수 있음.

\begin{equation}
\begin{split}
p(C_k | \textbf{x}) & \propto p(C_k)p(\textbf{x}|C_k) \\
& \propto p(C_k, x_1, ..., x_n)
\end{split}
\end{equation}

* 위의 식을 연쇄 법칙을 사용해 다음과 같이 쓸 수 있음.

\begin{equation}
\begin{split}
p(C_k, x_1, ..., x_n) & = p(C_k)p(x_1, ..., x_n | C_k) \\
& = p(C_k)p(x_1 | C_k)p(x_2, ..., x_n | C_k, x_1) \\
& = p(C_k)p(x_1 | C_k)p(x_2 | C_k, x_1)p(x_3, ..., x_n | C_k, x_1, x_2) \\
& = p(C_k)p(x_1 | C_k)p(x_2 | C_k, x_1)...p(x_n | C_k, x_1, x_2, ..., x_{n-1})
\end{split}
\end{equation}

* 나이브 베이즈 분류기는 모든 특성이 독립이라고 가정하기 때문에 위의 식을 다음과 같이 쓸 수 있음.

\begin{equation}
\begin{split}
p(C_k, x_1, ..., x_n) & \propto p(C_k)p(x_1|C_k)p(x_2|C_k)...p(x_n|C_k) \\
& \propto p(C_k) \prod_{i=1}^{n} p(x_i|C_k)
\end{split}
\end{equation}

* 위의 식을 통해 나온 값들 중 가장 큰 값을 갖는 클래스가 예측 결과

\begin{equation}
\hat{y} = \underset{k}{\arg\max} \; p(C_k) \prod_{i=1}^{n} p(x_i|C_k)
\end{equation}

In [3]:
weather = ['Sunny','Sunny','Overcast','Rainy','Rainy','Rainy','Overcast',
         'Sunny','Sunny','Rainy','Sunny','Overcast','Overcast','Rainy']
temp    = ['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']
play    = ['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']

df              = pd.DataFrame( [weather, temp] ).T
df.columns      = ['날씨', '온도']
df['외출 여부'] = play
df

,날씨,온도,외출 여부
0,Sunny,Hot,No
1,Sunny,Hot,No
2,Overcast,Hot,Yes
3,Rainy,Mild,Yes
4,Rainy,Cool,Yes
5,Rainy,Cool,No
6,Overcast,Cool,Yes
7,Sunny,Mild,No
8,Sunny,Cool,Yes
9,Rainy,Mild,Yes


In [4]:
x_data = df.iloc[:, :-1 ]
y_data = df.iloc[:, -1 ]
ct     = make_column_transformer( (OneHotEncoder(), ['날씨', '온도']) )
result = ct.fit_transform( df )
result

array([[0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1.]])

In [5]:
ct.get_feature_names()

['onehotencoder__x0_Overcast',
 'onehotencoder__x0_Rainy',
 'onehotencoder__x0_Sunny',
 'onehotencoder__x1_Cool',
 'onehotencoder__x1_Hot',
 'onehotencoder__x1_Mild']

In [6]:
testdf = pd.DataFrame( [['Sunny', 'Hot']], columns=['날씨', '온도'] )
testdf

,날씨,온도
0,Sunny,Hot


In [7]:
model_pipe = make_pipeline( ct, BernoulliNB() )
model_pipe.fit( x_data, y_data )
model_pipe.predict( testdf )

array(['No'], dtype='<U3')

- wine data set을 나이즈 베이즈를 이용하여 test[0] 데이터에 대한 분류를 출력하고 Score를 구하시오.

In [8]:
wine = load_wine()
x_train, x_test, y_train, y_test = train_test_split( wine.data, wine.target, test_size=0.25, stratify=wine.target  )
model_wine = make_pipeline( StandardScaler(), GaussianNB() )   # target 2진 초과의 다항 범주(0, 1, 2)
model_wine.fit( x_train, y_train )

print( f' Actual  : { y_test[0]}' )
print( f' Predict : { float(model_wine.predict( [x_test[0]] ) ) }' )
print( f' Score   : { round( model_wine.score( x_test, y_test )*100, 3) }%' )

 Actual  : 2
 Predict : 2.0
 Score   : 100.0%
